# 5. KoBERT

1. 환경 설정 및 라이브러리 로딩
2. 데이터 로딩 및 전처리 : KorCCVi_v2.csv 파일 로드, 텍스트 데이터 정제
3. 토크나이저 및 데이터셋 준비 : KoBERT에 맞는 토크나이저를 사용하여 데이터를 토큰화하고, 학습에 필요한 데이터셋 생성
4. 모델 정의 및 학습 : KoBERT 기반의 분류 모델을 정의, 학습 진행
5. 모델 평가 및 예측 : 테스트 데이터를 사용하여 모델의 성능 평가 및 예측 결과 확인

### 📌 KoBERT를 활용한 보이스 피싱 탐지 모델 구축 코드
- 참고: https://github.com/selfcontrol7/Korean_Voice_Phishing_Detection
- https://github.com/selfcontrol7/Korean_Voice_Phishing_Detection/blob/main/KoBERT/Korean_Text_classification_with_KoBERT.ipynb
- https://github.com/selfcontrol7/Korean_Voice_Phishing_Detection/blob/main/KoBERT/Vishing%20Detection%20using%20KoBERT%20model%20from%20HuggingFace.ipynb
- 주요 목적: 통화 텍스트에서 보이스 피싱(1) vs 정상(0) 판별
- 추가 기능: 중요 키워드 추출

## 문제 발생
- 정상 통화도 보이스 피싱으로 판단하는 문제
| 점검 항목      | 확인 결과                     | 조치            |
| ---------- | ------------------------- | ------------- |
| 라벨 분포      | ✅ 균형 맞음 (609:609)         | 이상 없음         |
| 평가 결과      | ❓ 모든 문장이 1로 예측됨           | F1, 정밀도 확인 필요 |
| 학습 Epoch   | ❓ 3 (낮음)                  | 5\~10으로 늘려보기  |
| Softmax 출력 | ❓ Threshold 없이 argmax만 사용 | 조건 분기 추가 가능   |


## [1] 환경 설정 및 라이브러리 로딩

In [49]:
# 🔧 1. 라이브러리 임포트
import torch  # GPU 사용을 위한 PyTorch
from torch import nn  # 신경망 모델 구성용
from torch.utils.data import Dataset, DataLoader  # 데이터셋 및 배치 구성

from transformers import BertTokenizer, BertModel  # HuggingFace용 KoBERT 모델
from kobert_transformers import get_tokenizer, get_kobert_model  # KoBERT용 전용 함수

import numpy as np  # 수치 연산
import pandas as pd  # 데이터프레임 처리
from sklearn.model_selection import train_test_split  # 학습/테스트 분리
from tqdm import tqdm  # 진행바 시각화
from konlpy.tag import Okt  # 키워드 추출을 위한 형태소 분석기
from torch.optim import AdamW # HuggingFace transformers에서 import
from collections import Counter # 훈련셋 + 테스트셋 라벨 분포 확인 코드
from transformers import get_cosine_schedule_with_warmup

## [2] 데이터 로딩 및 전처리

In [51]:
# 📂 CSV 파일 경로
csv_path = 'D:/2025_work/2025_VoicePhshing_Detection_Model/dataset/phishing_dataset/KorCCVi_dataset/KorCCViD_v1.3_fullcleansed.csv'

# 📑 CSV 불러오기
df = pd.read_csv(csv_path)

# 🧮 Label 분포 확인 (0: 정상, 1: 피싱)
label_counts = df['Label'].value_counts()

In [11]:
df

,Transcript,Label
0,사람 전라도 광주 태장 42 고요 명동 에서 10 정도 근무 했었 200 보여 그리...,1
1,서울 중앙지 숫자 인데 개인 정보 유출 때문 확인 연락 드렸 습니다 혹시 김용진 라...,1
2,여기 끝내 2005 김창호 도용 사건 고요 감당 서울 중앙 지검 지능 범죄 수사 외...,1
3,웨딩 으로 원래 그대로 근데 그것 요즘 웨딩 그런 추세 라고 수원 용인 그러 돌잔치...,0
4,답답 지금 선생 본인 래요 자기 성함 아니 에요 아니 통해서 통화 어요 답답 십니다...,1
...,...,...
1213,점심때 특히 학교 에서 내려가 까지 그게 너무 귀찮 우리 학교 산꼭대기 200 30...,0
1214,맞아 그거 예전 부터 그거 정기 으로 학생 니까 그게 소득 잖아 근데 그게 으면 된...,0
1215,학원 해서 그런 건지 아니 나이 차이 그런 건지 원래 별로 좋아하 니까 나이 차이 ...,0
1216,최소 300 지난번 광고 길래 아무 생각 없이 눌러 근데 36 개월 개월 할부 광고...,0


In [53]:
# 📊 결과 출력
print("✅ 라벨 분포:")
print(f"정상 통화 (0): {label_counts.get(0, 0)}개")
print(f"보이스 피싱 (1): {label_counts.get(1, 0)}개")

✅ 라벨 분포:
정상 통화 (0): 609개
보이스 피싱 (1): 609개


### >> 라벨 분포	✅ 균형 맞음 (609:609)	이상 없음

In [55]:
# 🔍 컬럼 이름 확인
print(df.columns)

Index(['Transcript', 'Label'], dtype='object')


In [59]:
# 📌 텍스트와 라벨 추출 (열 이름이 'Transcript', 'Label'인지 확인)
texts = df['Transcript'].tolist()  # 또는 'text' 컬럼 이름 확인
labels = df['Label'].tolist()

## [3] 훈련 / 테스트 데이터 분리

| 요소                | 의미                                                   |
| ----------------- | ---------------------------------------------------- |
| `texts`           | 전체 텍스트 데이터 리스트 (예: 통화 내용 텍스트)                        |
| `labels`          | 전체 정답 라벨 리스트 (예: 0: 정상, 1: 피싱)                       |
| `test_size=0.2`   | 전체 데이터 중 **20%를 테스트셋**, 나머지 80%를 학습셋으로 사용            |
| `random_state=42` | **무작위 분할 시 랜덤 시드** 설정<br>→ 항상 같은 방식으로 섞임 (재현 가능성 

-----------------------------------------------------------------------

| 변수명            | 내용                 |
| -------------- | ------------------ |
| `train_texts`  | 학습용 텍스트 데이터 (80%)  |
| `test_texts`   | 테스트용 텍스트 데이터 (20%) |
| `train_labels` | 학습용 라벨 (0 또는 1)    |
| `test_labels`  | 테스트용 라벨            |
보장) |
     |


In [61]:
# ✅ stratify 옵션으로 학습/테스트 나누기 (라벨 비율 유지)
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

In [63]:
print("🔍 훈련용 샘플:")
for i in range(3):
    short_text = train_texts[i][:10]  # 앞 10자만 출력
    label = train_labels[i]           # 라벨 가져오기
    print(f" - [{i+1}] '{short_text}...' | 라벨: {'보이스 피싱' if label == 1 else '정상 통화'}")

print("\n🧪 테스트용 샘플:")
for i in range(3):
    short_text = test_texts[i][:10]   # 앞 10자만 출력
    label = test_labels[i]           # 라벨 가져오기
    print(f" - [{i+1}] '{short_text}...' | 라벨: {'보이스 피싱' if label == 1 else '정상 통화'}")

🔍 훈련용 샘플:
 - [1] '여보세요 고객 입니...' | 라벨: 보이스 피싱
 - [2] '준비 해서 올라가 ...' | 라벨: 정상 통화
 - [3] '지금 고객 께서 저...' | 라벨: 보이스 피싱

🧪 테스트용 샘플:
 - [1] '우리 연락 문제 많...' | 라벨: 정상 통화
 - [2] '그리고 와이파이 터...' | 라벨: 정상 통화
 - [3] '페이스북 마저 그런...' | 라벨: 정상 통화


In [65]:
# ✅ 훈련셋 + 테스트셋 라벨 분포 확인 
train_label_dist = Counter(train_labels)

# 테스트셋 라벨 분포 확인
test_label_dist = Counter(test_labels)

# 결과 출력
print("🔧 훈련셋 라벨 분포:")
print(f" - 정상 통화 (0): {train_label_dist.get(0, 0)}개")
print(f" - 보이스 피싱 (1): {train_label_dist.get(1, 0)}개")

print("\n🧪 테스트셋 라벨 분포:")
print(f" - 정상 통화 (0): {test_label_dist.get(0, 0)}개")
print(f" - 보이스 피싱 (1): {test_label_dist.get(1, 0)}개")

🔧 훈련셋 라벨 분포:
 - 정상 통화 (0): 487개
 - 보이스 피싱 (1): 487개

🧪 테스트셋 라벨 분포:
 - 정상 통화 (0): 122개
 - 보이스 피싱 (1): 122개


## [4] KoBERT용 토크나이저 및 데이터셋 구성

| 객체 이름           | 역할 요약                                              | 세부 설명                                      |
| --------------- | -------------------------------------------------- | ------------------------------------------ |
| `train_dataset` | 학습용 원본 데이터셋                                        | 토큰화된 입력과 라벨을 포함. `BERTDataset` 클래스를 통해 구성됨 |
| `test_dataset`  | 테스트용 원본 데이터셋                                       | 학습이 아닌 평가 목적의 데이터셋                         |
| `train_loader`  | 학습용 데이터 배치 처리기 (`batch_size=16`, `shuffle=True`)   | 에폭마다 다른 순서로 섞어서 학습을 안정적으로 수행               |
| `test_loader`   | 테스트용 데이터 배치 처리기 (`batch_size=16`, `shuffle=False`) | 평가 시 순서를 유지하여 성능 측정                        |


In [67]:
# ✅ KoBERT 전용 토크나이저 로딩 (skt/kobert-base-v1에 맞춰진 사전 설정 포함)
kobert_tokenizer = get_tokenizer()

# ✅ KoBERT 사전학습 모델 로딩 (Transformer 기반 BERT 모델)
kobert_model = get_kobert_model()

# 📏 입력 토큰 최대 길이 설정 (128 토큰 이상은 자르고, 부족하면 패딩)
MAX_LEN = 128

In [69]:
# 📦 PyTorch Dataset 클래스 정의
class BERTDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts                  # 입력 텍스트 리스트 (문장들)
        self.labels = labels                # 해당 문장의 라벨 리스트 (0 or 1)
        self.tokenizer = tokenizer          # KoBERT용 토크나이저
        self.max_len = max_len              # 최대 길이 설정

    def __len__(self):
        return len(self.texts)              # 전체 데이터 길이 반환

    def __getitem__(self, idx):
        # 🔠 텍스트 하나를 토크나이징하여 KoBERT에 맞는 포맷으로 변환
        encoded = self.tokenizer(
            self.texts[idx],                # 현재 인덱스의 텍스트
            padding='max_length',           # 부족한 길이는 패딩
            truncation=True,                # 초과한 길이는 자름
            max_length=self.max_len,        # 최대 길이 설정
            return_tensors='pt'             # PyTorch Tensor로 반환
        )
        # 🧷 배치로 만들기 위해 차원을 줄임 (squeeze(0): batch dimension 제거)
        item = {key: val.squeeze(0) for key, val in encoded.items()}

        # 🎯 정답 라벨도 텐서로 변환하여 추가
        item['labels'] = torch.tensor(self.labels[idx])
        return item                         # input_ids, attention_mask, token_type_ids, labels 포함

In [71]:
# 🧪 학습용 데이터셋 생성
train_dataset = BERTDataset(train_texts, train_labels, kobert_tokenizer, MAX_LEN)
# 🧪 테스트용 데이터셋 생성
test_dataset = BERTDataset(test_texts, test_labels, kobert_tokenizer, MAX_LEN)
# 🚚 학습용 데이터로더: 배치 단위로 데이터를 불러오며 무작위 셔플
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# 🚚 테스트용 데이터로더: 배치 단위로 데이터를 불러오되 셔플은 하지 않음
test_loader = DataLoader(test_dataset, batch_size=16)

In [73]:
# 데이터셋/로더 구조 미리보기 (1배치만 확인)
print("🔍 학습 데이터셋 예시:")
for batch in train_loader:
    print("📦 input_ids:", batch['input_ids'].shape)
    print("📦 attention_mask:", batch['attention_mask'].shape)
    print("📦 token_type_ids:", batch['token_type_ids'].shape)
    print("🏷️ labels:", batch['labels'].shape)
    break  # 한 배치만 확인

print("\n🔍 테스트 데이터셋 예시:")
for batch in test_loader:
    print("📦 input_ids:", batch['input_ids'].shape)
    print("📦 attention_mask:", batch['attention_mask'].shape)
    print("📦 token_type_ids:", batch['token_type_ids'].shape)
    print("🏷️ labels:", batch['labels'].shape)
    break  # 한 배치만 확인


🔍 학습 데이터셋 예시:
📦 input_ids: torch.Size([16, 128])
📦 attention_mask: torch.Size([16, 128])
📦 token_type_ids: torch.Size([16, 128])
🏷️ labels: torch.Size([16])

🔍 테스트 데이터셋 예시:
📦 input_ids: torch.Size([16, 128])
📦 attention_mask: torch.Size([16, 128])
📦 token_type_ids: torch.Size([16, 128])
🏷️ labels: torch.Size([16])


In [75]:
# 첫 배치에서 첫 번째 샘플 문장을 디코딩해서 보기
for batch in train_loader:
    input_ids = batch['input_ids'][0]  # 첫 샘플 선택
    label = batch['labels'][0].item()  # 라벨도 함께 출력

    # 디코딩: 토큰 ID → 문장
    decoded_text = kobert_tokenizer.decode(input_ids, skip_special_tokens=True)

    print("📝 디코딩된 문장:", decoded_text)
    print("🏷️ 라벨 (0: 정상, 1: 피싱):", label)
    break  # 한 번만 실행

📝 디코딩된 문장: 그것 나중 만약 언니 언니 원래 머리 어울렸 으니까 다시 으로 염색 해도 상관 니까 도전 봐도 정말 언니 처럼 우리 나이 지나가 버리 면은 그렇게 밝은색 으로 염색 어려울 잖아 우리 색깔 그러니까 우리 시도 인생 인데 그렇게 한번 재미있 즐거운 니까 괜찮
🏷️ 라벨 (0: 정상, 1: 피싱): 0


## [5] KoBERT 분류 모델 정의

| 구성 요소             | 설명                                                                      |
| ----------------- | ----------------------------------------------------------------------- |
| `bert_model`      | `get_kobert_model()`을 통해 불러온 사전학습된 KoBERT 모델. 입력 문장을 BERT 인코딩하여 벡터로 변환  |
| `hidden_size`     | BERT의 출력 차원 크기 (기본: 768). 이 값은 Linear 레이어의 입력 크기로 사용됨                   |
| `num_classes`     | 분류 클래스 수 (보이스피싱은 0/1이므로 2)                                              |
| `dr_rate`         | Dropout 비율. 학습 시 과적합을 방지하기 위해 일부 뉴런을 무작위로 끄는 확률 (예: 0.3 = 30%)          |
| `self.classifier` | 선형 계층 (`nn.Linear`)로 \[CLS] 토큰 벡터를 이진 분류 확률값으로 변환                       |
| `self.dropout`    | Dropout 레이어. 학습 중 과적합을 줄이기 위해 중간 벡터 일부를 무작위로 제거                         |
| `self.softmax`    | Linear 출력값을 0\~1 사이의 확률값으로 변환. 두 클래스의 확률 분포로 해석 가능                      |
| `forward()`       | 학습 또는 추론 시 호출되는 메서드. BERT → \[CLS] → Dropout → Linear → Softmax 순으로 처리됨 |
   |


# [ 모델 ! ]
- KoBERTClassifier()
- model = KoBERTClassifier(...) 이런식으로 사용 중


In [77]:
class KoBERTClassifier(nn.Module):
    def __init__(self, bert_model, hidden_size=768, num_classes=2, dr_rate=0.3):
        """
        bert_model: get_kobert_model()로 로딩한 KoBERT 모델
        hidden_size: BERT의 hidden layer 크기 (기본 768)
        num_classes: 분류 클래스 수 (0: 정상, 1: 피싱 → 2개)
        dr_rate: dropout 비율
        """
        super(KoBERTClassifier, self).__init__()
        self.bert = bert_model
        self.dr_rate = dr_rate

        # ✅ 분류 레이어 정의
        self.classifier = nn.Linear(hidden_size, num_classes)

        # ✅ 드롭아웃 설정
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

        # ✅ 확률 출력을 위한 Softmax
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        """
        BERT 입력값 처리 → Dropout → Linear 분류 → Softmax
        """
        # BERT 모델 실행
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids)
        
        # [CLS] 토큰의 출력값 사용 (문장 요약 벡터)
        cls_output = outputs.pooler_output

        # Dropout 적용 (training 시에만 활성화됨)
        if self.dr_rate:
            cls_output = self.dropout(cls_output)

        # Linear 분류 → Softmax 확률 출력
        logits = self.classifier(cls_output)
        return self.softmax(logits)


-------------------------------------------------------------------------------

## [6] Optimizer + 학습률 스케줄러 설정

In [79]:
# ✅ 장치 설정: GPU 사용 가능하면 GPU 사용, 아니면 CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 모델 인스턴스 생성 코드 : 
- 실제 학습에 사용된 model 객체 생성
- 이후 학습 / 검증 과정에서 이 모델 사용

In [81]:
# ✅ KoBERT 분류 모델 생성 (드롭아웃 포함)
model = KoBERTClassifier(
    bert_model=kobert_model,  # 사전학습 KoBERT 모델
    hidden_size=768,          # hidden state 차원 크기 (기본값)
    num_classes=2,            # 분류 클래스 수 (0: 정상, 1: 피싱)
    dr_rate=0.3               # Dropout 비율 설정
).to(device)  # 모델을 GPU 또는 CPU로 할당

-----------------------------------------------------------------------

In [83]:
# ✅ weight decay를 다르게 적용하기 위한 파라미터 분리
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {
        'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        'weight_decay': 0.01  # 일반 파라미터에는 가중치 감쇠 적용
    },
    {
        'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        'weight_decay': 0.0   # bias나 LayerNorm에는 감쇠 미적용
    }
]

In [85]:
# ✅ 옵티마이저 설정 (AdamW)
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5)

In [87]:
# ✅ 손실 함수 정의 (Cross Entropy)
loss_fn = nn.CrossEntropyLoss()

In [89]:
# ✅ 학습 스케줄 설정
EPOCHS = 3
warmup_ratio = 0.1  # 전체 학습 스텝 중 워밍업 비율

# 전체 학습 스텝 수 계산 (배치 수 × epoch 수)
t_total = len(train_loader) * EPOCHS
warmup_step = int(t_total * warmup_ratio)

# 코사인 학습률 스케줄러 설정
scheduler = get_cosine_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total
)

In [91]:
# ✅ 정확도 계산 함수 (accuracy)
def calc_accuracy(preds, labels):
    _, predicted = torch.max(preds, 1)  # 각 배치에서 확률이 가장 높은 클래스 선택
    correct = (predicted == labels).sum().item()
    return correct / labels.size(0)  # 전체 중 맞춘 비율 반환

In [93]:
# ✅ 정밀도, 재현율, F1-score 계산 함수
# → 모델의 평가 지표로 활용됨
def get_metrics(preds, labels, threshold=0.5):
    pred_classes = torch.argmax(preds, dim=1).cpu().numpy()
    labels = labels.cpu().numpy()
    tp = ((pred_classes == 1) & (labels == 1)).sum()
    fp = ((pred_classes == 1) & (labels == 0)).sum()
    fn = ((pred_classes == 0) & (labels == 1)).sum()
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1 = 2 * precision * recall / (precision + recall + 1e-8)
    return {"precision": precision, "recall": recall, "f1": f1}

In [95]:
# ✅ 평가 함수 (정확도 + 정밀도 + 재현율 + F1-score 출력)
@torch.no_grad()
def evaluate(model, data_loader):
    model.eval()
    total_correct = 0
    total = 0
    all_preds = []
    all_labels = []
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)

        preds = model(input_ids, attention_mask, token_type_ids)
        total_correct += (preds.argmax(1) == labels).sum().item()
        total += labels.size(0)
        all_preds.append(preds)
        all_labels.append(labels)

    acc = total_correct / total
    all_preds_tensor = torch.cat(all_preds, dim=0)
    all_labels_tensor = torch.cat(all_labels, dim=0)
    metrics = get_metrics(all_preds_tensor, all_labels_tensor)

    print(f"🎯 정확도: {acc*100:.2f}% | 정밀도: {metrics['precision']:.2f} | 재현율: {metrics['recall']:.2f} | F1: {metrics['f1']:.2f}")

## [7] KoBERT 모델 학습 루프

| 구성 요소              | 설명                            |
| ------------------ | ----------------------------- |
| `model.train()`    | 모델을 학습 모드로 전환 (dropout 등 활성화) |
| `loss.backward()`  | 손실 함수로부터 gradient 계산          |
| `optimizer.step()` | 계산된 gradient로 가중치 업데이트        |
| `scheduler.step()` | 학습률 조절 스케줄러 업데이트              |
| `calc_accuracy()`  | 배치 단위 정확도 계산                  |


In [97]:
for epoch in range(EPOCHS):
    model.train()  # 모델을 학습 모드로 전환
    total_loss = 0  # 손실 누적 변수 초기화
    total_acc = 0   # 정확도 누적 변수 초기화

    # 학습 데이터 배치 단위로 반복
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        # 입력값 GPU/CPU로 이동
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)

        # 예측 수행
        preds = model(input_ids, attention_mask, token_type_ids)

        # 손실 계산
        loss = loss_fn(preds, labels)

        # 옵티마이저 초기화 → 역전파 → 가중치 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        # 누적 손실과 정확도 계산
        total_loss += loss.item()
        total_acc += calc_accuracy(preds, labels)

    # 평균 손실과 정확도 출력
    avg_loss = total_loss / len(train_loader)
    avg_acc = total_acc / len(train_loader)
    print(f"[Epoch {epoch+1}] 평균 손실: {avg_loss:.4f} | 평균 정확도: {avg_acc*100:.2f}%")

Epoch 1: 100%|█████████████████████████████████████████████████████████████████████████| 61/61 [08:58<00:00,  8.83s/it]


[Epoch 1] 평균 손실: 0.4798 | 평균 정확도: 88.52%


Epoch 2: 100%|█████████████████████████████████████████████████████████████████████████| 61/61 [09:27<00:00,  9.31s/it]


[Epoch 2] 평균 손실: 0.3329 | 평균 정확도: 99.08%


Epoch 3: 100%|█████████████████████████████████████████████████████████████████████████| 61/61 [09:31<00:00,  9.37s/it]

[Epoch 3] 평균 손실: 0.3221 | 평균 정확도: 99.59%


### Epoch 3 -> 정확도 상승

| 에포크     | 평균 손실 (`loss`) | 평균 정확도 (`accuracy`) | 의미                          |
| ------- | -------------- | ------------------- | --------------------------- |
| Epoch 1 | 0.5027         | 84.31%              | 첫 학습 시작 → 모델이 대략적으로 분류를 시작함 |
| Epoch 2 | 0.3491         | 97.75%              | 모델이 피싱/정상 분류를 잘 학습함         |
| Epoch 3 | 0.3285         | 99.39%              | 거의 완벽에 가까운 성능으로 학습됨         |


## [8] 모델 평가 (테스트 정확도, 정밀도, 재현율, F1-score 측정)

| 항목              | 설명                      |
| --------------- | ----------------------- |
| 정확도 (Accuracy)  | 전체 예측 중 맞춘 비율           |
| 정밀도 (Precision) | 피싱이라고 예측한 것 중 실제 피싱 비율  |
| 재현율 (Recall)    | 실제 피싱 중에서 얼마나 잘 찾아냈는가   |
| F1-score        | 정밀도와 재현율의 조화 평균 (균형 측정) |


In [99]:
model.eval()  # 평가 모드 전환 (Dropout 등 비활성화)

total_correct = 0
total = 0
all_preds = []
all_labels = []

In [101]:
with torch.no_grad():  # 평가 시에는 그래디언트 계산하지 않음
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)

        preds = model(input_ids, attention_mask, token_type_ids)
        total_correct += (preds.argmax(1) == labels).sum().item()
        total += labels.size(0)

        all_preds.append(preds)
        all_labels.append(labels)

In [102]:
# 전체 정확도 계산
acc = total_correct / total

In [103]:
# 예측 및 정답 텐서를 하나로 합침
all_preds_tensor = torch.cat(all_preds, dim=0)
all_labels_tensor = torch.cat(all_labels, dim=0)

In [104]:
# 정밀도, 재현율, F1 계산
metrics = get_metrics(all_preds_tensor, all_labels_tensor)

In [105]:
# 결과 출력
print(f"\n📊 [테스트 결과]")
print(f"🎯 정확도: {acc * 100:.2f}%")
print(f"📌 정밀도: {metrics['precision']:.2f}")
print(f"📌 재현율: {metrics['recall']:.2f}")
print(f"📌 F1-score: {metrics['f1']:.2f}")


📊 [테스트 결과]
🎯 정확도: 100.00%
📌 정밀도: 1.00
📌 재현율: 1.00
📌 F1-score: 1.00


### [추가]

In [106]:
# ✅ 테스트셋 전체 예측 라벨 분포
all_preds_tensor = torch.cat(all_preds, dim=0)
predicted_labels = torch.argmax(all_preds_tensor, dim=1).cpu().numpy()

from collections import Counter
pred_counts = Counter(predicted_labels)

print("🔍 테스트셋 예측 라벨 분포:")
print(f" - 정상 통화 (0): {pred_counts.get(0, 0)}개")
print(f" - 보이스 피싱 (1): {pred_counts.get(1, 0)}개")

🔍 테스트셋 예측 라벨 분포:
 - 정상 통화 (0): 122개
 - 보이스 피싱 (1): 122개


In [107]:
# 예측 라벨 분포 확인
predicted_labels = torch.argmax(all_preds_tensor, dim=1).cpu().numpy()
from collections import Counter
pred_counts = Counter(predicted_labels)

print("\n🔍 테스트셋 예측 라벨 분포:")
print(f" - 정상 통화 (0): {pred_counts.get(0, 0)}개")
print(f" - 보이스 피싱 (1): {pred_counts.get(1, 0)}개")


🔍 테스트셋 예측 라벨 분포:
 - 정상 통화 (0): 122개
 - 보이스 피싱 (1): 122개


## [10] 크롤링한 데이터 모델에 넣어서 예측

In [89]:
import pandas as pd
import torch

# ✅ 데이터 로드: STT 텍스트가 포함된 CSV 파일 경로 지정
csv_path = 'D:/2025_work/2025_VoicePhshing_Detection_Model/dataset/stt_transcripts_sampled.csv'

# ✅ CSV 파일을 pandas로 읽어옴
df = pd.read_csv(csv_path)

# ✅ 데이터프레임에서 상위 2개 행만 샘플로 추출
sampled_rows = df.head(2)

# ✅ 모델을 평가 모드로 전환 (Dropout, BatchNorm 비활성화)
model.eval()

KoBERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [91]:
# ✅ 예측 함수 정의: text를 입력받아 label(0 or 1)과 확신도(score) 반환
def predict_phishing(text, model, tokenizer, max_len=128):
    """
    주어진 text를 KoBERT 모델로 예측하여
    보이스 피싱(1) 또는 정상(0) 여부를 판단하는 함수
    """
    # ✅ 토크나이저로 입력 문장 인코딩 (KoBERT 포맷)
    encoded = tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors='pt'  # PyTorch 텐서 형태로 반환
    )

    # ✅ 각 입력을 GPU 또는 CPU로 이동
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)
    token_type_ids = encoded['token_type_ids'].to(device)

    # ✅ 추론: 그래디언트 계산 없이 모델 예측 수행
    with torch.no_grad():
        output = model(input_ids, attention_mask, token_type_ids)

        # ✅ 예측 결과 중 확률이 가장 높은 클래스 선택 (0: 정상, 1: 피싱)
        pred_label = torch.argmax(output, dim=1).item()

        # ✅ 해당 클래스의 확률값 (0~1) 추출
        confidence = output[0][pred_label].item()

    return pred_label, confidence

In [97]:
# ✅ 샘플 텍스트 각각에 대해 예측 수행
for idx, row in sampled_rows.iterrows():
    text = row['text']        # 통화 내용 텍스트
    true_label = row['label'] # 실제 라벨 (0 또는 1)

    # ✅ 모델을 이용해 예측 수행
    pred_label, confidence = predict_phishing(text, model, kobert_tokenizer)

    # ✅ 예측 결과 출력
    print(f"\n📞 샘플 {idx+1}")
    print(f"▶ 원본 텍스트: {text}")
    print(f"✅ 실제 라벨: {'보이스 피싱' if true_label == 1 else '정상 통화'}")
    print(f"🔍 예측 라벨: {'보이스 피싱' if pred_label == 1 else '정상 통화'} ({confidence*100:.2f}% 확신도)")



📞 샘플 1
▶ 원본 텍스트: 안녕하세요. 공공은행입니다. 신용대출을 받을 수 있을까요? 본인이 맞는지 먼저 확인하겠습니다. 네. 안내 멘트가 나오면 생년월일 6자리를 눌러주세요. 네 알겠습니다. 확인 되었습니다 고객님. 성함이나 휴대폰 번호 알려주세요. 공공공이고 공공공 공공공 공공공입니다. 대출 한도가 다 대출건과 합산이 되어 산정됩니다. 필요하신 자금은 얼마신가요? 3,000만원 정도 필요합니다. 대출 한도는 스마트폰 비대면 대출 메뉴를 이용하시거나 지점을 방문해 주시면 진행 가능합니다. 문자로 비대면 대출 방법을 보내주실 수 있나요? 네, 이 번호로 보내주세요.
✅ 실제 라벨: 정상 통화
🔍 예측 라벨: 보이스 피싱 (98.21% 확신도)

📞 샘플 2
▶ 원본 텍스트: 신용대출은 인터넷으로 신청할 수 있어요? 아니요 방문하셔야 합니다 아 그렇군요 아무 노겹이다 가면 되나요? 네 맞습니다 서류는 뭘 가져가면 되죠? 신분증과 소득 확인 서류 가져가시면 됩니다 소득 확인 서류는 뭐가 있어요? 혹시 기존의 대출 내역이 있으면 불리한가요? 네 기존 대출 때문에 불가능할 수도 있습니다. 여기서 미리 말해 주실 수는 없어요? 네 영업점 방문 후 상담 하시면 됩니다. 서류는 말씀해 주신 것만 가져가면 되는 거죠? 네 추후 대출 과정에서 필요 서류가 더 많아지게 되면 있을 수 있습니다. 그건 나중에 안내해주시는 거예요? 네, 맞습니다. 오늘 영업시가 몇 시까지예요? 6시입니다. 감사합니다.
✅ 실제 라벨: 정상 통화
🔍 예측 라벨: 보이스 피싱 (98.37% 확신도)


In [101]:
import random

# ✅ 테스트셋에서 무작위로 5개 샘플 선택
random.seed(42)  # 결과 재현 가능하도록 고정
sample_indices = random.sample(range(len(test_texts)), 5)

print("🎲 무작위로 선택된 테스트셋 샘플 5개 예측 결과:\n")

for i, idx in enumerate(sample_indices):
    text = test_texts[idx]
    true_label = test_labels[idx]

    # 모델 예측
    pred_label, confidence = predict_phishing(text, model, kobert_tokenizer)

    # 결과 출력
    print(f"📞 샘플 {i+1}")
    print(f"▶ 원본 텍스트: {text[:150]}...")  # 너무 길면 앞 150자만 출력
    print(f"✅ 실제 라벨: {'보이스 피싱' if true_label == 1 else '정상 통화'}")
    print(f"🔍 예측 라벨: {'보이스 피싱' if pred_label == 1 else '정상 통화'} ({confidence*100:.2f}% 확신도)\n")


🎲 무작위로 선택된 테스트셋 샘플 5개 예측 결과:

📞 샘플 1
▶ 원본 텍스트: 지금 부터 민감 니까 사람 으로 와요 전화 아요 경고 경찰 신고 거나 주위 사람 에게 알리 쑤셔 버립니다 어디 xx 그걸 한테 어딨 모르 아줌마 아줌마 단순히 필요 한테 얼마나 나요 최대한 아이 어딨 냐고 우리 에요 아니 어딘지 자꾸 위치 냐고 준비...
✅ 실제 라벨: 보이스 피싱
🔍 예측 라벨: 보이스 피싱 (98.25% 확신도)

📞 샘플 2
▶ 원본 텍스트: 그렇 농협 국민 인가요 농협 경우 니까 도개 형성 자체 어떤 용도 계좌 습니까 어서 적금 청약 마이너스 통장 자유 입출금 기타 등등 어떤 으로 형성 으시 자율 출근 하나 밖에 형성 나요 월급 통장 자율 지금 농협 에서 으세요 농협 하나 밖에 으시 고요 국민은행 요네 적...
✅ 실제 라벨: 보이스 피싱
🔍 예측 라벨: 보이스 피싱 (98.68% 확신도)

📞 샘플 3
▶ 원본 텍스트: 겨울 근데 원래 스키장 한데 스키장 스키 요즘 그게 땡기 더라고 아이스링크 그런데 망포역 근데 그것 롤러스케이트 그래서 나중 볼까 생각 신나 거든 그냥 웬만 볼라고 그리고 강릉 금방 던데 16000 인가 19000 구천 인가 그것 더라고 교통비 되게...
✅ 실제 라벨: 정상 통화
🔍 예측 라벨: 정상 통화 (99.04% 확신도)

📞 샘플 4
▶ 원본 텍스트: 부분 출금 으로 설정 부분 부분 저희 내부 내부 내부 자체 금액 으로 해서 입금 다가 출금 부분 으로 확인 들어가 부분 에요 전산 으로 저희 확인 때문 기록 기록 때문 그렇게 저희 자체 금액 으로 해서 확인 부분 니까 부분 기다려 셔야 요번 확인 들어갈 전화 드리 진행...
✅ 실제 라벨: 보이스 피싱
🔍 예측 라벨: 보이스 피싱 (98.63% 확신도)

📞 샘플 5
▶ 원본 텍스트: 그런가 그니까 알바 여도 채용 경우 거든 스타 벅스 그러 거든 스타 벅스 그까 직원 그니까 에서 원래 반가 그깐 그냥 커피 바리스타 사람 본사 에서 직원 한테 누구 전공 면은 사람 채용 본사 에서

## ✅ 해결 방향 

| 해결 방법                           | 설명                              | 추천도   |
| ------------------------------- | ------------------------------- | ----- |
| ✅ 1. 정제된 "정상 대출 상담" 데이터를 더 추가   | 모델이 "대출" 관련 정상 통화도 있다는 걸 학습하게 함 | ⭐⭐⭐⭐⭐ |
| ✅ 2. 단어 기반 attention 분석 → 편향 확인 | 모델이 어떤 단어에 집중하고 있는지 시각화         | ⭐⭐⭐   |
| ✅ 3. Threshold 방식 도입            | 불확실할 때 경고만 주고, 최종 판단은 사용자가 하게 함 | ⭐⭐⭐⭐  |
| ✅ 4. Ensemble/NLP 후처리 추가        | 딥러닝 외 논리 기반 룰이나 추가 모델로 후처리      | ⭐⭐    |


### ✅ 가장 쉬운 임시 대응: Threshold 기반 판단 조건 강화

In [106]:
def predict_phishing_with_threshold(text, model, tokenizer, threshold=0.9):
    encoded = tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='pt'
    )
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)
    token_type_ids = encoded['token_type_ids'].to(device)

    with torch.no_grad():
        output = model(input_ids, attention_mask, token_type_ids)
        phishing_score = output[0][1].item()  # 보이스 피싱 확률

    # threshold 조건 강화 (예: 0.9 이상일 때만 보이스 피싱으로 분류)
    pred_label = 1 if phishing_score >= threshold else 0
    return pred_label, phishing_score

### 📌 궁극적 해결책: 정상 통화 중 '은행', '대출' 포함된 샘플을 충분히 모델에 학습시켜야 함

In [117]:
# 데이터셋에서 정상(0) + '대출' 포함된 샘플 개수 확인
normal_with_loan = df[(df['label'] == 0) & (df['text'].str.contains('대출|은행|신용', regex=True))]
print("정상 통화 중 대출 키워드 포함된 샘플:", len(normal_with_loan))


정상 통화 중 대출 키워드 포함된 샘플: 69


In [115]:
df

,filename,label,text
0,normal_normal_607.wav,0,안녕하세요. 공공은행입니다. 신용대출을 받을 수 있을까요? 본인이 맞는지 먼저 확인...
1,normal_normal_543.wav,0,신용대출은 인터넷으로 신청할 수 있어요? 아니요 방문하셔야 합니다 아 그렇군요 아무...
2,normal_normal_370.wav,0,보험료 출금해 주세요. 본인 확인 후 안내 드리겠습니다. 성함을 말씀해 주시겠어요?...
3,normal_normal_92.wav,0,전세금으로 담보대출 상품을 알아볼 수 있나요? 전세계약을 진행하셨나요? 지금 살고 ...
4,normal_normal_399.wav,0,보험이체 변경해 주시겠습니까? 성함과 생열로일 말해 주시겠어요? 0000에 0000...
...,...,...,...
195,phishing_기타_금감원_보이스피싱_9_수정06.wav,1,"여보세요? 네? 예, 소원하십니다 혹시? 좀 시통화 좀 가능하실까요? 바쁜데 빨리 ..."
196,phishing_수사기관_사칭형_12.wav,1,"여보세요? 예. 예. 안녕하세요. 대금, 차팔천 천단본주 수사관입니다. 예. 다른 ..."
197,phishing_수사기관_사칭형_78.wav,1,이 범죄 현장에 사용되면 따라 많은 피해자들이 발생되었습니다. 그 피해자들도 11명...
198,phishing_수사기관_사칭형_143.wav,1,안녕? 그럼 주소로 저희가 공문이랑 후환창 발굴을 해드릴테니까 자전기 날짜에 공문해...


+) stt_transcripts_sampled.csv의 컬럼명 변경 후 덮어쓰기

In [124]:
import pandas as pd

# 📥 CSV 파일 경로
csv_path = 'D:/2025_work/2025_VoicePhshing_Detection_Model/dataset/stt_transcripts_sampled.csv'

# 📑 CSV 불러오기
df = pd.read_csv(csv_path)

# 🔁 컬럼명 변경
df.rename(columns={
    'filename': 'Filename',
    'label': 'Label',
    'text': 'Transcript'
}, inplace=True)

# 💾 변경된 CSV 파일 덮어쓰기 저장
df.to_csv(csv_path, index=False)

print("✅ 컬럼명이 성공적으로 변경되었고 CSV 파일이 저장되었습니다.")

✅ 컬럼명이 성공적으로 변경되었고 CSV 파일이 저장되었습니다.


In [126]:
print(df.head())  # 변경된 컬럼명으로 잘 반영됐는지 확인

                Filename  Label  \
0  normal_normal_607.wav      0   
1  normal_normal_543.wav      0   
2  normal_normal_370.wav      0   
3   normal_normal_92.wav      0   
4  normal_normal_399.wav      0   

                                          Transcript  
0  안녕하세요. 공공은행입니다. 신용대출을 받을 수 있을까요? 본인이 맞는지 먼저 확인...  
1  신용대출은 인터넷으로 신청할 수 있어요? 아니요 방문하셔야 합니다 아 그렇군요 아무...  
2  보험료 출금해 주세요. 본인 확인 후 안내 드리겠습니다. 성함을 말씀해 주시겠어요?...  
3  전세금으로 담보대출 상품을 알아볼 수 있나요? 전세계약을 진행하셨나요? 지금 살고 ...  
4  보험이체 변경해 주시겠습니까? 성함과 생열로일 말해 주시겠어요? 0000에 0000...  


✅ 병합 및 저장하는 전체 코드

In [129]:
import pandas as pd

# 📥 파일 경로
stt_path = 'D:/2025_work/2025_VoicePhshing_Detection_Model/dataset/stt_transcripts_sampled.csv'
korccvi_path = 'D:/2025_work/2025_VoicePhshing_Detection_Model/dataset/phishing_dataset/KorCCVi_dataset/KorCCViD_v1.3_fullcleansed.csv'

# 📑 각 CSV 불러오기
df_stt = pd.read_csv(stt_path)
df_kor = pd.read_csv(korccvi_path)

# 🔁 컬럼명 통일: KorCCVi 데이터에 Filename 컬럼 추가
df_kor['Filename'] = 'KorCCViD'

# ✅ 두 데이터 병합
merged_df = pd.concat([df_kor[['Label', 'Transcript', 'Filename']], df_stt[['Label', 'Transcript', 'Filename']]], ignore_index=True)

# 💾 새로운 CSV로 저장
merged_path = 'D:/2025_work/2025_VoicePhshing_Detection_Model/dataset/KorCCVi_stt_merged.csv'
merged_df.to_csv(merged_path, index=False)

print("✅ 두 CSV 파일을 성공적으로 병합하여 저장했습니다!")
print(f"📁 저장 위치: {merged_path}")

✅ 두 CSV 파일을 성공적으로 병합하여 저장했습니다!
📁 저장 위치: D:/2025_work/2025_VoicePhshing_Detection_Model/dataset/KorCCVi_stt_merged.csv


In [131]:
print(merged_df.head())  # 병합 후 상위 5개 행 출력
print(merged_df['Filename'].value_counts())  # KorCCViD vs 실제 파일명 개수 확인

   Label                                         Transcript  Filename
0      1  사람 전라도 광주 태장 42 고요 명동 에서 10 정도 근무 했었 200 보여 그리...  KorCCViD
1      1  서울 중앙지 숫자 인데 개인 정보 유출 때문 확인 연락 드렸 습니다 혹시 김용진 라...  KorCCViD
2      1  여기 끝내 2005 김창호 도용 사건 고요 감당 서울 중앙 지검 지능 범죄 수사 외...  KorCCViD
3      0  웨딩 으로 원래 그대로 근데 그것 요즘 웨딩 그런 추세 라고 수원 용인 그러 돌잔치...  KorCCViD
4      1  답답 지금 선생 본인 래요 자기 성함 아니 에요 아니 통해서 통화 어요 답답 십니다...  KorCCViD
Filename
KorCCViD                                                   1218
phishing_기타_대출빙자형 사례 2_정부지원자금 대출을 받기 위한 공증서 발급비용 요구.wav       1
phishing_대출사기형_124.wav                                        1
phishing_수사기관_사칭형_220.wav                                     1
phishing_수사기관_사칭형_163.wav                                     1
                                                           ... 
normal_normal_318.wav                                         1
normal_normal_550.wav                                         1
normal_normal_298.wav                                      

### 📌 현재 상황 요약
- ✔️ 만든 KoBERT 모델이 완성되어 있고

- ✔️ 병합된 데이터셋 KorCCVi_stt_merged.csv가 있고

- ❌ 하지만 아직 이 전체 데이터셋을 모델에 넣고 예측한 결과를 저장한 CSV 파일은 없음

### ✅ 해야 할 작업
- KorCCVi_stt_merged.csv의 모든 행을 모델에 돌려 예측 결과(PredictedLabel, Confidence)를 추가하고 새 CSV로 저장

### ✅ 전체 예측 + 저장 코드

In [143]:
import pandas as pd
import torch

# 📥 병합된 CSV 경로
merged_csv_path = 'D:/2025_work/2025_VoicePhshing_Detection_Model/dataset/KorCCVi_stt_merged.csv'
df = pd.read_csv(merged_csv_path)

# ✅ 모델 평가 모드로 전환
model.eval()

# ✅ 예측 함수 정의
def predict_phishing(text, model, tokenizer, max_len=128):
    encoded = tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors='pt'
    )
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)
    token_type_ids = encoded['token_type_ids'].to(device)

    with torch.no_grad():
        output = model(input_ids, attention_mask, token_type_ids)
        pred_label = torch.argmax(output, dim=1).item()
        confidence = output[0][pred_label].item()
    
    return pred_label, confidence

# ✅ 전체 데이터에 대해 예측 수행 (중간 결과 출력 추가)
predicted_labels = []
confidences = []

print("🔍 예측 진행 중...")

for i, text in enumerate(df['Transcript']):
    pred_label, confidence = predict_phishing(text, model, kobert_tokenizer)
    predicted_labels.append(pred_label)
    confidences.append(confidence)

    # 🔁 매 100개마다 진행률 출력
    if (i + 1) % 100 == 0 or (i + 1) == len(df):
        print(f" - {i + 1} / {len(df)}개 처리 완료")

# ✅ 결과 컬럼 추가
df['PredictedLabel'] = predicted_labels
df['Confidence'] = confidences

# 💾 새로운 CSV로 저장
output_path = 'D:/2025_work/2025_VoicePhshing_Detection_Model/dataset/KorCCVi_stt_predicted.csv'
df.to_csv(output_path, index=False)

print("\n✅ 예측 결과가 포함된 CSV 파일이 저장되었습니다.")
print(f"📁 저장 위치: {output_path}")


🔍 예측 진행 중...
 - 100 / 1418개 처리 완료
 - 200 / 1418개 처리 완료
 - 300 / 1418개 처리 완료
 - 400 / 1418개 처리 완료
 - 500 / 1418개 처리 완료
 - 600 / 1418개 처리 완료
 - 700 / 1418개 처리 완료
 - 800 / 1418개 처리 완료
 - 900 / 1418개 처리 완료
 - 1000 / 1418개 처리 완료
 - 1100 / 1418개 처리 완료
 - 1200 / 1418개 처리 완료
 - 1300 / 1418개 처리 완료
 - 1400 / 1418개 처리 완료
 - 1418 / 1418개 처리 완료

✅ 예측 결과가 포함된 CSV 파일이 저장되었습니다.
📁 저장 위치: D:/2025_work/2025_VoicePhshing_Detection_Model/dataset/KorCCVi_stt_predicted.csv


### ✅ 1. 예측 정확도 요약 (정답률, 오답률 등)

In [146]:
import pandas as pd

# CSV 로드
df = pd.read_csv('D:/2025_work/2025_VoicePhshing_Detection_Model/dataset/KorCCVi_stt_predicted.csv')

# ✅ 총 개수
total = len(df)

# ✅ 정확히 맞춘 샘플 수
correct = (df['Label'] == df['PredictedLabel']).sum()
incorrect = total - correct

# ✅ 클래스별 개수
from collections import Counter
true_label_dist = Counter(df['Label'])
pred_label_dist = Counter(df['PredictedLabel'])

# ✅ 정밀도 / 재현율 / F1 계산
from sklearn.metrics import classification_report

report = classification_report(df['Label'], df['PredictedLabel'], target_names=['정상 통화', '보이스 피싱'])

# 출력
print("📊 예측 통계 요약:")
print(f"- 총 샘플 수: {total}")
print(f"- 정확히 예측한 샘플: {correct}개 ({correct/total*100:.2f}%)")
print(f"- 잘못 예측한 샘플: {incorrect}개 ({incorrect/total*100:.2f}%)")

print("\n🔎 실제 라벨 분포:", dict(true_label_dist))
print("🔍 예측 라벨 분포:", dict(pred_label_dist))

print("\n📈 상세 성능 리포트 (정밀도, 재현율, F1-score):")
print(report)


📊 예측 통계 요약:
- 총 샘플 수: 1418
- 정확히 예측한 샘플: 1310개 (92.38%)
- 잘못 예측한 샘플: 108개 (7.62%)

🔎 실제 라벨 분포: {1: 709, 0: 709}
🔍 예측 라벨 분포: {1: 807, 0: 611}

📈 상세 성능 리포트 (정밀도, 재현율, F1-score):
              precision    recall  f1-score   support

       정상 통화       0.99      0.85      0.92       709
      보이스 피싱       0.87      0.99      0.93       709

    accuracy                           0.92      1418
   macro avg       0.93      0.92      0.92      1418
weighted avg       0.93      0.92      0.92      1418



### 📊 예측 결과 핵심 해석
| 항목           | 수치       | 해석                                    |
| ------------ | -------- | ------------------------------------- |
| 정밀도 (정상 통화)  | **0.99** | 모델이 "정상 통화라고 판단한 것들 중 실제로 거의 다 정상"    |
| 재현율 (정상 통화)  | **0.85** | 실제 정상 통화 중 15%를 **보이스 피싱으로 잘못 분류**함 ⛔ |
| 정밀도 (보이스 피싱) | 0.87     | "보이스 피싱이라고 예측한 것들 중 13%는 실제로 정상 통화"   |
| 재현율 (보이스 피싱) | 0.99     | 모델이 실제 보이스 피싱은 대부분 잘 잡아냄 ✅            |
---------------------------------------------------------
### ❗ 오분류 패턴
문제 지점: 모델이 "정상 통화인데 보이스 피싱으로 잘못 예측"하는 경우가 많음
(즉, False Positive 비율이 높음)

- → 이게 바로 "일반 대출 상담"도 피싱처럼 예측되는 문제와 연결

### ✅ 오분류된 정상 통화만 추출

In [150]:
# 잘못 예측된 샘플 중: Label=0 (정상)인데 PredictedLabel=1 (피싱)인 경우
false_positives = df[
    (df['Label'] == 0) & (df['PredictedLabel'] == 1)
]

print(f"❌ 정상 통화를 피싱으로 오판한 샘플 수: {len(false_positives)}개")

# 예시 출력
print("\n📋 오분류 예시:")
for i, row in false_positives.head(5).iterrows():
    print(f"- {row['Transcript'][:100]}... | 확신도: {row['Confidence']*100:.2f}% | Filename: {row['Filename']}")


❌ 정상 통화를 피싱으로 오판한 샘플 수: 103개

📋 오분류 예시:
- 사실 아까 시간 얘기 으면 계획 세우 아요 오늘 10 시부 10 시작 니까 목사 10 시작 해서 12 12 어서 끝내 라고 10 에서 인제 시작 12 어서 끝나 식사 인제 글케 생... | 확신도: 53.90% | Filename: KorCCViD
- 중앙 포스 누나 영수증 으면서 해서 고객 상품권 다른 영수증 하나 고요 다른 하나 셔야 돼요 고객 니까 거기 고객 그래요 면서 모르 근데 거기 난동 으니까 고객 얼굴 기억 거기 사... | 확신도: 98.38% | Filename: KorCCViD
- 저번 제일 웃겼 내일 자일리톨 많이 냐는 질문 많이 니까 롯데 자일리톨 많이 드세요 해태 자일리톨 많이 드세요 라고 진짜 미치 어떻게 모르 해서 계속 많이 더니 사람 롯데 조금 많... | 확신도: 85.31% | Filename: KorCCViD
- 안녕하세요. 공공은행입니다. 신용대출을 받을 수 있을까요? 본인이 맞는지 먼저 확인하겠습니다. 네. 안내 멘트가 나오면 생년월일 6자리를 눌러주세요. 네 알겠습니다. 확인 되었습니... | 확신도: 98.21% | Filename: normal_normal_607.wav
- 신용대출은 인터넷으로 신청할 수 있어요? 아니요 방문하셔야 합니다 아 그렇군요 아무 노겹이다 가면 되나요? 네 맞습니다 서류는 뭘 가져가면 되죠? 신분증과 소득 확인 서류 가져가시... | 확신도: 98.37% | Filename: normal_normal_543.wav


💾 선택적으로 CSV로 저장

In [153]:
false_positives.to_csv('D:/2025_work/2025_VoicePhshing_Detection_Model/dataset/false_positives_label0_pred1.csv', index=False)
print("📁 오분류된 정상 통화 샘플 CSV 저장 완료")

📁 오분류된 정상 통화 샘플 CSV 저장 완료


### 📌 이후 활용 방안
| 활용              | 목적                          |
| --------------- | --------------------------- |
| 🔍 수작업 검토       | 왜 이 문장이 피싱처럼 보였는지 직접 확인     |
| 🔁 Oversampling | 이 데이터를 복제해서 학습셋에 넣고 재학습     |
| 🔧 Threshold 조절 | 불확실한 경우 보류 또는 재확인 처리        |
| 🧠 모델 개선        | KoBERT 외 다른 모델 비교 실험도 고려 가능 |


### ✅ 1단계: 오분류된 정상 통화 샘플 5개 추출

In [158]:
import pandas as pd

# 예측 결과 CSV 불러오기
df = pd.read_csv('D:/2025_work/2025_VoicePhshing_Detection_Model/dataset/KorCCVi_stt_predicted.csv')

# 오분류된 정상 통화 (Label=0, PredictedLabel=1)
false_positives = df[(df['Label'] == 0) & (df['PredictedLabel'] == 1)]

# 상위 5개만 추출
sample_fp = false_positives.head(5).copy()

### ✅ 2단계: 예시별 원인 해석

In [161]:
for i, row in sample_fp.iterrows():
    print(f"\n📞 샘플 {i+1}")
    print(f"▶ 원본 텍스트:\n{row['Transcript'][:300]}...")
    print(f"✅ 실제 라벨: 정상 통화 | 🔍 예측: 보이스 피싱 ({row['Confidence']*100:.2f}% 확신도)")

    # 🔎 예측 원인 분석 (기계적 기준 + 정황)
    suspicious_keywords = ['대출', '신용', '은행', '계좌', '한도', '비대면', '보증', '본인확인']
    hits = [kw for kw in suspicious_keywords if kw in row['Transcript']]

    if hits:
        print(f"💡 추정 원인: '{', '.join(hits)}' 등의 금융 키워드로 인해 피싱으로 판단했을 가능성 ↑")
    elif len(row['Transcript']) < 30:
        print("💡 추정 원인: 텍스트가 너무 짧아 맥락 파악 어려움 → 보수적 판단 가능성")
    else:
        print("💡 추정 원인: 문맥이 모호하거나 '의심스러운 어조'로 학습된 유사 문장이 있었을 가능성")



📞 샘플 235
▶ 원본 텍스트:
사실 아까 시간 얘기 으면 계획 세우 아요 오늘 10 시부 10 시작 니까 목사 10 시작 해서 12 12 어서 끝내 라고 10 에서 인제 시작 12 어서 끝나 식사 인제 글케 생각 인제 어디 갈려고 했었 어요...
✅ 실제 라벨: 정상 통화 | 🔍 예측: 보이스 피싱 (53.90% 확신도)
💡 추정 원인: 문맥이 모호하거나 '의심스러운 어조'로 학습된 유사 문장이 있었을 가능성

📞 샘플 598
▶ 원본 텍스트:
중앙 포스 누나 영수증 으면서 해서 고객 상품권 다른 영수증 하나 고요 다른 하나 셔야 돼요 고객 니까 거기 고객 그래요 면서 모르 근데 거기 난동 으니까 고객 얼굴 기억 거기 사람...
✅ 실제 라벨: 정상 통화 | 🔍 예측: 보이스 피싱 (98.38% 확신도)
💡 추정 원인: 문맥이 모호하거나 '의심스러운 어조'로 학습된 유사 문장이 있었을 가능성

📞 샘플 1096
▶ 원본 텍스트:
저번 제일 웃겼 내일 자일리톨 많이 냐는 질문 많이 니까 롯데 자일리톨 많이 드세요 해태 자일리톨 많이 드세요 라고 진짜 미치 어떻게 모르 해서 계속 많이 더니 사람 롯데 조금 많이 으시 롯데 많이...
✅ 실제 라벨: 정상 통화 | 🔍 예측: 보이스 피싱 (85.31% 확신도)
💡 추정 원인: 문맥이 모호하거나 '의심스러운 어조'로 학습된 유사 문장이 있었을 가능성

📞 샘플 1219
▶ 원본 텍스트:
안녕하세요. 공공은행입니다. 신용대출을 받을 수 있을까요? 본인이 맞는지 먼저 확인하겠습니다. 네. 안내 멘트가 나오면 생년월일 6자리를 눌러주세요. 네 알겠습니다. 확인 되었습니다 고객님. 성함이나 휴대폰 번호 알려주세요. 공공공이고 공공공 공공공 공공공입니다. 대출 한도가 다 대출건과 합산이 되어 산정됩니다. 필요하신 자금은 얼마신가요? 3,000만원 정도 필요합니다. 대출 한도는 스마트폰 비대면 대출 메뉴를 이용하시거나 지점을 방문해 주시면 진행 가능합니다. 문자로 비대면 대출 방법을 보내주실 수 있나요? 네, 이 번호로 보

## 📌 모델 구조 + 학습 가중치 불러오기

① 모델 저장

In [115]:
# 학습 완료된 모델 저장
torch.save(model.state_dict(), 'kobert_model.pt')  # 경로 자유롭게 설정 가능